In [34]:
from qiskit.circuit.library import QFT
from disqco.circuits.cp_fraction import cp_fraction, cz_fraction
from qiskit import transpile
from QASMBench.interface.qiskit import QASMBenchmark
from disqco.graphs.quantum_network import QuantumNetwork


num_qubits = 32

circuit = cp_fraction(num_qubits, num_qubits, fraction=0.5)

# circuit = QFT(num_qubits, do_swaps=False)

num_partitions = 4  # Define the number of partitions or QPUs you will distribute over
depth = circuit.depth()

qpu_size = num_qubits // num_partitions + 1 # Define the number of qubits per QPU. For simplicity, we divide qubits evenly and add one additional space to each for teleportation (an extra space is needed for the teleportation qubit so the algorithmr requires it!)
qpu_sizes = [qpu_size] * num_partitions # Store the number of qubits per QPU in a list

# Create a quantum network with the specified number of qubits and partitions

# If we do not specificy the connectivity, we have all-to-all by default.

quantum_network = QuantumNetwork(qpu_sizes)



basis_gates = ['u', 'cp']

# Transpile the circuit to the basis gates
circuit = transpile(circuit, basis_gates=basis_gates)

print(f'Number of qubits in circuit {circuit.num_qubits}')
print(f'Circuit depth: {circuit.depth()}')

Number of qubits in circuit 32
Circuit depth: 32


In [35]:
from disqco.parti.FM.fiduccia import *

FM_partitioner = FiducciaMattheyses(circuit=circuit,
                                    network=quantum_network)

In [36]:
results = FM_partitioner.partition()

print(f'Best cost: {results['best_cost']}')

Best cost: 100


In [37]:
from disqco.parti.genetic.genetic_algorithm_beta import *

GA_partitioner = GeneticPartitioner(circuit=circuit, network=quantum_network, group_gates=True, log=True, multi_process=True)

results = GA_partitioner.partition()
print(f'Best cost: {results['best_cost']}')

Best cost: 126


In [38]:
from disqco.graphs.coarsening.coarsener import HypergraphCoarsener

coarsener = HypergraphCoarsener()

coarseneing_method = coarsener.coarsen_recursive_batches_mapped

results = FM_partitioner.multilevel_partition(coarsener=coarseneing_method)

Best cost at level 0: 112
Best cost at level 1: 108
Best cost at level 2: 101
Best cost at level 3: 96
Best cost at level 4: 94
Best cost at level 5: 94


In [39]:
print(f'Best cost: {results['best_cost']}')

Best cost: 94


In [40]:
results = GA_partitioner.multilevel_partition(coarsener=coarseneing_method)

AttributeError: 'GeneticPartitioner' object has no attribute 'num_qubits'

In [ ]:
from disqco.graphs.quantum_network import linear_coupling

coupling = linear_coupling(num_partitions)

network = QuantumNetwork(qpu_sizes, qpu_connectivity=coupling)
FM_partitioner = FiducciaMattheyses(circuit=circuit,
                                    network=network)


results = FM_partitioner.partition(log=True)

Initial cost: 189
All passes complete.
Final cost: 144


In [ ]:
results = FM_partitioner.multilevel_partition()

Best cost at level 0: 147
Best cost at level 1: 145
Best cost at level 2: 133
Best cost at level 3: 129
Best cost at level 4: 127
Best cost at level 5: 125
